In [ ]:
# pip install selenium
# pip install webdriver-manager

In [ ]:
# import the necessary packages

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup


import pandas as pd
import time
import dotenv
import os
import lxml

In [ ]:
# load in username and password using a .env file
dotenv.load_dotenv()
login_email = os.getenv('cqi_username')
login_password = os.getenv('cqi_password')

In [ ]:
# ROBUSTA COFFEE BEANS

# it should open each link one by one and get the data and make 5 files
# if it takes to long, selenium will not be able to parse it and will have an incomplete file (this is okay,
# it just won't be included in the final dataset)

# should run in one go

# open chromedriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
time.sleep(2)

# navigate to login page
driver.get('https://database.coffeeinstitute.org/login')
time.sleep(3)

# submit login credentials
form = wait.until(EC.presence_of_element_located((By.XPATH, '//html/body/content[@class="scrollable"]/div[@class="container page"]/div[@class="form short"]/div[@class="login panel"]/form'))) # NOTE: find_element_by_* has been removed

# form = driver.find_element('xpath', '//html/body/content[@class="scrollable"]/div[@class="container page"]/div[@class="form short"]/div[@class="login panel"]/form') # NOTE: find_element_by_* has been removed
username = driver.find_element('name', "username")
password = driver.find_element('name', "password")
time.sleep(2)

username.send_keys(login_email)
password.send_keys(login_password)
driver.find_element('class name', "submit").click()
time.sleep(2)


# navigate to coffees page, then to arabicas page containing links to all quality reports
# coffees = driver.find_element('xpath', '//html/body/header/nav[@id="main"]/div[@class="container"]/div[@class="in"]/a[@href="/coffees"]').click()
coffees = wait.until(EC.presence_of_element_located((By.XPATH, '//html/body/header/nav[@id="main"]/div[@class="container"]/div[@class="in"]/a[@href="/coffees"]'))).click()#

time.sleep(5)
driver.find_element('link text', 'Robusta Coffees').click()
time.sleep(3)

# these values can be changed if this breaks midway through collecting data to pick up close to where you left off
page = 0
coffeenum = 0

while True:
	print('page {}'.format(page))

	# 50 rows in these tables * 7 columns per row = 350 cells. Every 7th cell clicks through to that coffee's data page
	for i in range(1, 400, 8):
		time.sleep(2)

		# paginate back to the desired page number
		# don't think there's a way around this - the back() option goes too far back
		# some page numbers aren't available in the ui, but 'next' always is unless you've reached the end
		for p_num in range(page):
			page_buttons = driver.find_elements('class name', 'paginate_button')
			page_buttons[-1].click() # the 'next' button
			time.sleep(1)
			page_buttons = driver.find_elements('class name', 'paginate_button')

		# select the cell to click through to the next coffee-data page
		time.sleep(2) # this next line errors out sometimes, maybe it needs more of a time buffer
		test_page = driver.find_elements('xpath', '//td')[i].click()
		time.sleep(2)
		print('rows: ')
		print(len(driver.find_elements('xpath', "//tr")))
		tables = driver.find_elements(By.TAG_NAME, "table")

		# loop over all coffee reports on the page, processing each one and writing to csv
		print('tables: ')
		print(len(tables))
		j = 0
		for tab in tables:
			try:
				t = BeautifulSoup(tab.get_attribute('outerHTML'), "html.parser")
				#print(t)
				df = pd.read_html(str(t))
				name = 'coffee_{}_table_{}.csv'.format(coffeenum, j)
				df[0].to_csv(name)
				print(name)
			except:
				# only one's needed but I want this to be obnoxious since it's the only way I'm logging this currently
				print('ERROR: {} failed'.format(name))
				print('ERROR: {} failed'.format(name))
				print('ERROR: {} failed'.format(name))
				print('ERROR: {} failed'.format(name))
			j += 1

		# go back to page with all other coffee results
		#driver.back() # note: this isn't working as expected, manually going back to pg 1 via url instead
		driver.get('https://database.coffeeinstitute.org/coffees/robusta')
		time.sleep(2)
		coffeenum += 1

	page += 1
	if page == 2:
		break


# close the driver
driver.close()

In [ ]:
import os
import pandas as pd
import numpy as np

# Change this to your working directory
folder_path = '/Users/ashik/UVA/ds 6021 project/robusta_final'  # or arabica
os.chdir(folder_path)

# List all files in directory
all_files = os.listdir()

# Keep only valid files matching coffee_#_table_#.csv
coffee_list = (
    pd.DataFrame({'files': all_files})
    .query("files.str.match(r'^coffee_\\d+_table_\\d+\\.csv$')", engine='python')
    .reset_index(drop=True)
)

# Check how many valid coffee files were found
print(f"Found {len(coffee_list)} valid coffee files.")
if len(coffee_list) == 0:
    raise ValueError("No valid coffee_#_table_#.csv files found. Check your folder path.")

# Split filenames to identify coffee IDs and tables
coffee_list_split_temp = coffee_list['files'].str.split(pat='_', n=2, expand=True)
coffee_list_split_temp.columns = ['coffee', 'id', 'tables']

# Combine parts for easier grouping
coffee_list_split = pd.DataFrame({
    'coffee': coffee_list_split_temp['coffee'] + '_' + coffee_list_split_temp['id'],
    'tables': coffee_list_split_temp['tables']
})

# Count tables per coffee
table_counts = coffee_list_split.groupby('coffee').count()
print("Coffees missing some tables:")
print(table_counts[table_counts['tables'] != 5])

# Identify which coffee IDs to skip
incomplete = table_counts[table_counts['tables'] != 5].index.tolist()
skips = [int(c.split('_')[1]) for c in incomplete]
print("Skipping:", skips)

df_list = []

for i in range(210):
    if i in skips:
        print(f"Skipping {i}")
        continue

    try:
        df0 = pd.read_csv(f'coffee_{i}_table_0.csv')
        df1 = pd.read_csv(f'coffee_{i}_table_1.csv')
        df2 = pd.read_csv(f'coffee_{i}_table_2.csv')
        df3 = pd.read_csv(f'coffee_{i}_table_3.csv')
        df4 = pd.read_csv(f'coffee_{i}_table_4.csv')
    except FileNotFoundError:
        print(f"Missing file(s) for coffee_{i}")
        continue

    # Process df1
    df1.columns = ['zero', 'one', 'two', 'three', 'four']
    colnames1 = df1['one'].tolist()
    colnames2 = df1['three'].tolist()
    data1 = df1['two'].tolist()
    data2 = df1['four'].tolist()
    df1_processed = pd.DataFrame([(data1 + data2)], columns=(colnames1 + colnames2))

    # Process df2
    df2.columns = ['zero', 'one', 'two', 'three', 'four']
    colnames1 = df2['one'].tolist()
    colnames2 = df2['three'].tolist()
    data1 = df2['two'].tolist()
    data2 = df2['four'].tolist()
    df2_processed = pd.DataFrame([(data1 + data2)], columns=(colnames1 + colnames2))

    # Process df3
    df3.columns = ['zero', 'one', 'two', 'three', 'four']
    colnames1 = df3['one'].tolist()
    colnames2 = df3['three'].tolist()
    data1 = df3['two'].tolist()
    data2 = df3['four'].tolist()
    df3_processed = pd.DataFrame([(data1 + data2)], columns=(colnames1 + colnames2))

    # Process df4
    df4.columns = ['zero', 'one', 'two']
    colnames1 = df4['one'].tolist()
    data1 = df4['two'].tolist()
    df4_processed = pd.DataFrame([data1], columns=colnames1)

    if i > 1:
        prev_cols = df.columns

    df = pd.concat([df1_processed, df2_processed, df3_processed, df4_processed], axis=1)
    df = df.rename(columns={np.nan: "NA"})
    df_list.append(df)
    print(i)

    these_cols = df.columns
    if i > 1:
        pass

j = 0
for i in df_list:
    print(f"{j} shape: {i.shape}")
    j += 1

df_final = pd.concat(df_list, axis=0)
print(df_final.columns)
print(df_final.shape)
print(df_final.head())

df_final.to_csv('df_1_robusta.csv', index=False)

# figure out what coffee_89 is and have to manually to add it back
# https://database.coffeeinstitute.org/coffee/991448 is missing


In [ ]:
# ARABICA COFFEE BEANS

## this one will time out because of issues with selenium and memory space?
## if it times out, just start from the page that you were on (change the page = # in the while loop below)
## and continue from there

## if you need to pick up where you left off, make sure to put the current files in their own folder
## because it will overwrite them if you do not separate them

folder_path = '/Users/ashik/UVA/ds 6021 project/arabica_final'  # or arabica
os.chdir(folder_path)

# open chromedriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
time.sleep(2)

# navigate to login page
driver.get('https://database.coffeeinstitute.org/login')
time.sleep(3)

# submit login credentials
form = wait.until(EC.presence_of_element_located((By.XPATH, '//html/body/content[@class="scrollable"]/div[@class="container page"]/div[@class="form short"]/div[@class="login panel"]/form'))) # NOTE: find_element_by_* has been removed

# form = driver.find_element('xpath', '//html/body/content[@class="scrollable"]/div[@class="container page"]/div[@class="form short"]/div[@class="login panel"]/form') # NOTE: find_element_by_* has been removed
username = driver.find_element('name', "username")
password = driver.find_element('name', "password")
time.sleep(2)

username.send_keys(login_email)
password.send_keys(login_password)
driver.find_element('class name', "submit").click()
time.sleep(2)


# navigate to coffees page, then to arabicas page containing links to all quality reports
# coffees = driver.find_element('xpath', '//html/body/header/nav[@id="main"]/div[@class="container"]/div[@class="in"]/a[@href="/coffees"]').click()
coffees = wait.until(EC.presence_of_element_located((By.XPATH, '//html/body/header/nav[@id="main"]/div[@class="container"]/div[@class="in"]/a[@href="/coffees"]'))).click()#

time.sleep(5)
driver.find_element('link text', 'Arabica Coffees').click()
time.sleep(3)

# these values can be changed if this breaks midway through collecting data to pick up close to where you left off
page = 6
coffeenum = 0



while True:
	print('page {}'.format(page))

	# 50 rows in these tables * 7 columns per row = 350 cells. Every 7th cell clicks through to that coffee's data page
	for i in range(1, 400, 8):
		time.sleep(2)

		# paginate back to the desired page number
		# don't think there's a way around this - the back() option goes too far back
		# some page numbers aren't available in the ui, but 'next' always is unless you've reached the end
		for p_num in range(page):
			page_buttons = driver.find_elements('class name', 'paginate_button')
			page_buttons[-1].click() # the 'next' button
			time.sleep(1)
			page_buttons = driver.find_elements('class name', 'paginate_button')

		# select the cell to click through to the next coffee-data page
		time.sleep(2) # this next line errors out sometimes, maybe it needs more of a time buffer
		test_page = driver.find_elements('xpath', '//td')[i].click()
		time.sleep(2)
		print('rows: ')
		print(len(driver.find_elements('xpath', "//tr")))
		tables = driver.find_elements(By.TAG_NAME, "table")

		# loop over all coffee reports on the page, processing each one and writing to csv
		print('tables: ')
		print(len(tables))
		j = 0
		for tab in tables:
			try:
				t = BeautifulSoup(tab.get_attribute('outerHTML'), "html.parser")
				#print(t)
				df = pd.read_html(str(t))
				name = 'coffee_{}_table_{}.csv'.format(coffeenum, j)
				df[0].to_csv(name)
				print(name)
			except:
				# only one's needed but I want this to be obnoxious since it's the only way I'm logging this currently
				print('ERROR: {} failed'.format(name))
				print('ERROR: {} failed'.format(name))
				print('ERROR: {} failed'.format(name))
				print('ERROR: {} failed'.format(name))
			j += 1

		# go back to page with all other coffee results
		#driver.back() # note: this isn't working as expected, manually going back to pg 1 via url instead
		driver.get('https://database.coffeeinstitute.org/coffees/arabica')
		time.sleep(2)
		coffeenum += 1

	page += 1
	if page == 7: # or however many pages there are
		break


# close the driver
driver.close()

In [ ]:
import os
import pandas as pd
import numpy as np

# Change this to your working directory
folder_path = '/Users/ashik/UVA/ds 6021 project/arabica_final/page 7'  # or arabica
os.chdir(folder_path)

# List all files in directory
all_files = os.listdir()

# Keep only valid files matching coffee_#_table_#.csv
coffee_list = (
    pd.DataFrame({'files': all_files})
    .query("files.str.match(r'^coffee_\\d+_table_\\d+\\.csv$')", engine='python')
    .reset_index(drop=True)
)

# Check how many valid coffee files were found
print(f"Found {len(coffee_list)} valid coffee files.")
if len(coffee_list) == 0:
    raise ValueError("No valid coffee_#_table_#.csv files found. Check your folder path.")

# Split filenames to identify coffee IDs and tables
coffee_list_split_temp = coffee_list['files'].str.split(pat='_', n=2, expand=True)
coffee_list_split_temp.columns = ['coffee', 'id', 'tables']

# Combine parts for easier grouping
coffee_list_split = pd.DataFrame({
    'coffee': coffee_list_split_temp['coffee'] + '_' + coffee_list_split_temp['id'],
    'tables': coffee_list_split_temp['tables']
})

# Count tables per coffee
table_counts = coffee_list_split.groupby('coffee').count()
print("Coffees missing some tables:")
print(table_counts[table_counts['tables'] != 5])

# Identify which coffee IDs to skip
incomplete = table_counts[table_counts['tables'] != 5].index.tolist()
skips = [int(c.split('_')[1]) for c in incomplete]
print("Skipping:", skips)

df_list = []

for i in range(210):
    if i in skips:
        print(f"Skipping {i}")
        continue

    try:
        df0 = pd.read_csv(f'coffee_{i}_table_0.csv')
        df1 = pd.read_csv(f'coffee_{i}_table_1.csv')
        df2 = pd.read_csv(f'coffee_{i}_table_2.csv')
        df3 = pd.read_csv(f'coffee_{i}_table_3.csv')
        df4 = pd.read_csv(f'coffee_{i}_table_4.csv')
    except FileNotFoundError:
        print(f"Missing file(s) for coffee_{i}")
        continue

    # Process df1
    df1.columns = ['zero', 'one', 'two', 'three', 'four']
    colnames1 = df1['one'].tolist()
    colnames2 = df1['three'].tolist()
    data1 = df1['two'].tolist()
    data2 = df1['four'].tolist()
    df1_processed = pd.DataFrame([(data1 + data2)], columns=(colnames1 + colnames2))

    # Process df2
    df2.columns = ['zero', 'one', 'two', 'three', 'four']
    colnames1 = df2['one'].tolist()
    colnames2 = df2['three'].tolist()
    data1 = df2['two'].tolist()
    data2 = df2['four'].tolist()
    df2_processed = pd.DataFrame([(data1 + data2)], columns=(colnames1 + colnames2))

    # Process df3
    df3.columns = ['zero', 'one', 'two', 'three', 'four']
    colnames1 = df3['one'].tolist()
    colnames2 = df3['three'].tolist()
    data1 = df3['two'].tolist()
    data2 = df3['four'].tolist()
    df3_processed = pd.DataFrame([(data1 + data2)], columns=(colnames1 + colnames2))

    # Process df4
    df4.columns = ['zero', 'one', 'two']
    colnames1 = df4['one'].tolist()
    data1 = df4['two'].tolist()
    df4_processed = pd.DataFrame([data1], columns=colnames1)

    if i > 1:
        prev_cols = df.columns

    df = pd.concat([df1_processed, df2_processed, df3_processed, df4_processed], axis=1)
    df = df.rename(columns={np.nan: "NA"})
    df_list.append(df)
    print(i)

    these_cols = df.columns
    if i > 1:
        pass

j = 0
for i in df_list:
    print(f"{j} shape: {i.shape}")
    j += 1

df_final = pd.concat(df_list, axis=0)
print(df_final.columns)
print(df_final.shape)
print(df_final.head())

df_final.to_csv('df_7_arabica.csv', index=False)

# figure out what coffee_89 is and manually had to add it back


In [ ]:
folder_path = '/Users/ashik/UVA/ds 6021 project/arabica_final'  # or arabica
os.chdir(folder_path)

one = pd.read_csv('df_1to3_arabica.csv')
two = pd.read_csv('df_4_arabica.csv')
three = pd.read_csv('df_5-6_arabica.csv')
four = pd.read_csv('df_7_arabica.csv')

df_final = pd.concat([one, two, three, four], axis=0)
print(df_final.columns)
print(df_final.shape)
print(df_final.head())

df_final.to_csv('df_arabica.csv', index=False)